<a href="https://colab.research.google.com/github/csmsum/dl4img/blob/main/Practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Preparation

In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

lob_begin_red, log_begin_blue, log_begin_green = '\033[91m', '\033[94m', '\033[92m'
log_begin_bold, log_begin_underline = '\033[1m', '\033[4m'
log_end_format = '\033p0m'

##Neuron

In [37]:
class Neuron(object):
    def __init__(self, num_input, activation_function):
        super().__init__()
        self.w = np.random.uniform(size = num_input, low = -1., high = 1.)
        self.b = np.random.uniform(size = 1, low = -1., high = 1.)
        self.activation_function = activation_function

    def forward(self, x):
        z = np.dot(x, self.w)+self.b
        return self.activation_function(z)


```
class Neuron(object)
    super().__init__()
```
위와 같은 경우는, 파이썬 2의 잔재이다. 당시에는 class 내에 아무것도 들어가지 않을 때 object를 호출하고 object의 속성을 그대로 가져오는 super().__init__()을 함께 활성화했어야만 했다. 

In [38]:
input_size = 3
step_function = lambda y: 0 if y<=0 else 1
perceptron = Neuron(input_size, step_function)

##Regularization

In [2]:
num_classes = 10
img_rows, img_cols, img_ch = 28, 28, 1
input_shape = (img_rows, img_cols, img_ch)

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train, x_test = x_train/255.0, x_test/255.0

x_train = x_train.reshape(60000, 28, 28, 1)
x_test = x_test.reshape(10000, 28, 28, 1)

11501568/11490434 [==============================] - 0s 0us/step


In [3]:
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import (Input, Activation, Dense, Flatten, Conv2D,
                                     MaxPooling2D, Dropout, BatchNormalization)
epochs = 200
batch_size = 32

In [15]:
@tf.function
def conv_layer(x, kernels, bias, s):
    z = tf.nn.conv2d(x, kernels, strides = [1,s,s,1], padding = 'VALID') #여기서 [1,s,s,1]은 각각 [Number of inputs, Height, Width, Channel]인데, 이중 N,C는 1 by default이고 H,W는 strides 를 결정한다.
    return tf.nn.relu(z + bias)


class SimpleConvolutionLayer(tf.keras.layers.Layer): #add_weight 등의 methods들을 사용하기 위해서는 tf.keras.layers.Layer를 상속할 필요가 있다.
    def __init__(self, num_kernels = 32, kernel_size = (3,3), stride = 1):
        super().__init__()
        self.num_kernels = num_kernels
        self.kernel_size = kernel_size
        self.stride = stride

    def build(self, input_shape):
        num_input_ch = input_shape[-1] #이미지를 다룰 땐 BHWC 즉 [Batch_sz, Height, Width, Chanenl순으로 가는게 좋다.]
        kernels_shape = (*self.kernel_size, num_input_ch, self.num_kernels)
        """ㄴtuple이나 list앞의 *은 이들을 순서대로 풀어주는 역할을 한다. 예를 들어 (3,3)은 그냥 3, 3이 되어 나온다.
        즉 여기서 기본커널사이즈 33이라고 가정하고 mnist가 흑백이라는 것을 감안,
        또 커널 개수를 32라고 가정하면 (3,3,1,32)의 형태가 된다."""
        glorot_init = tf.initializers.GlorotUniform()
        self.kernels = self.add_weight(
            name = 'kernels', shape = kernels_shape,
            initializer = glorot_init, trainable = True)
        self.bias = self.add_weight(
            name = 'bias', shape = (self.num_kernels,),
            initializer = 'random_normal', trainable = True)

    def call(self, inputs):
        return conv_layer(inputs, self.kernels, self.bias, self.stride)
    
    def get_config(self):
        return {'num_kernels': self.num_kernels,
                'kernel_size': self.kernel_size,
                'stride': self.strides,
                'use_bias': self.use_bias}

In [16]:
from functools import partial

def l2_reg(coef = 1e-2):
    return lambda x: tf.reduce_sum(x**2) * coef

class ConvWithRegularizers(SimpleConvolutionLayer):
    def __init__(self, num_kernels = 32, kernel_size = (3,3), stride = 1,
        kernel_regularizer = l2_reg(), bias_regularizer = None):
        super().__init__(num_kernels, kernel_size, stride)
        self.kernel_regularizer = kernel_regularizer
        self.bias_regularizer = bias_regularizer

    def build(self, input_shape):
        super().build(input_shape)
        if self.kernel_regularizer is not None:
            self.add_loss(partial(self.kernel_regularizer, self.kernels))
        """partial 함수는 partial derivative를 가져와주는 함수가 아니다.
반대로, self.kernel_regularizer (l2_reg라는 이미 미분된 함수)에서 인수인 self.kernels를
이미 쓴 상태로 가져오겠다는 소리이다. 다시 말해서 self.kernels(regularizer함수)를 l2_reg(즉 미분)
에 넣은 결과값을 weight에 추가하겠다는 의미이다. """
        if self.bias_regularizer is not None:
            self.add_loss(partial(self.bias_regularizer, self.bias))

In [17]:
conv = ConvWithRegularizers(num_kernels = 32, kernel_size = (3,3), stride = 1,
                            kernel_regularizer = l2_reg(1.), bias_regularizer = l2_reg(1.))
conv.build(input_shape = tf.TensorShape((None, 28, 28, 1)))

reg_losses = conv.losses #여기서 conv.losses가 reg_losses로 이어질 수 있는 까닭은 아직 loss를 아무것도 설정하지 않은 상태에서 regularization만 써놨기 때문이다.
print('Regularization losses over kernel and bias parameters: {}'.format(
    [loss.numpy() for loss in reg_losses]))

kernel_norm, bias_norm = tf.reduce_sum(conv.kernels ** 2).numpy(), tf.reduce_sum(conv.bias ** 2).numpy()
print('L2 norms of kernel and bias parameters: {}'.format([kernel_norm, bias_norm]))


Regularization losses over kernel and bias parameters: [1.9206898, 0.120684355]
L2 norms of kernel and bias parameters: [1.9206898, 0.120684355]


In [18]:
model = Sequential([
    Input(shape=input_shape),
    ConvWithRegularizers(kernel_regularizer=l2_reg(1.), bias_regularizer=l2_reg(1.)),
    ConvWithRegularizers(kernel_regularizer=l2_reg(1.), bias_regularizer=l2_reg(1.)),
    ConvWithRegularizers(kernel_regularizer=l2_reg(1.), bias_regularizer=l2_reg(1.))  
])

print('Losses attached to the model and its layers:\n\r{} ({} losses)'.format(
    [loss.numpy() for loss in model.losses], len(model.losses)))

Losses attached to the model and its layers:
[1.8991263, 0.10561776, 31.978502, 0.068846345, 32.577663, 0.09254287] (6 losses)


In [23]:
class LeNet5(Model): #모델은 Layer과 그 확장판을 포함하고 있다. 아까 from tensorflow.keras.models import Model, Sequential
                        #에서 import 해놨다. 
    def __init__(self, num_classes,
             kernel_regularizer = l2_reg(), bias_regularizer = l2_reg()):
        super(LeNet5, self).__init__()
        self.conv1 = ConvWithRegularizers(
            6, kernel_size(5,5),
            kernel_regularizer = kernel_regularizer, bias_regularizer = bias_regularizer)
        self.conv2 = ConvWithRegularizers(
            16, kernel_size(5,5),
            kernel_regularizer = kernel_regularizer, bias_regularizer = bias_regularizer)
        self.max_pool = MaxPooling2D(pool_size = (2,2))
        self.flatten = Flatten()
        self.dense1 = Dense(120, activation = 'relu')
        self.dense2 = Dense(84, activation = 'relu')
        self.dense3 = Dense(num_classes, activation = 'softmax')

    def call(self, x):
        x = self.max_pool(self.conv1(x))
        x = self.max_pool(self.conv2(x))
        x = self.flatten(x)
        x = self.dense3(self.dense(self.dense1(x)))
        return x

In [ ]:
optimizer = tf.optimizers.SGD()
dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(batch_size)
log_string_template = 'Epoch {0:3}/{1}: main loss = {5}{2:5.3f}{6}; ' + \
                      'reg loss = {5}{3:5.3f}{6}; val acc = {5}{4:5.3f}%{6}'

def train_classifier_on_mnist(model, log_frequency = 10):
    avg_main_loss = tf.keras.metrics.Mean(name = 'avg_main_loss', dtype = tf.float32)
    avg_reg_loss = tf.keras.metrics.Mean(name = 'avg_reg_loss', dtype = tf.float32)    

    print("Training: {}start{}".format(log_begin_red, log_end_format))
    for epoch in range(epochs):
        for (batch_image, batch_gts) in dataset:

TypeError: ignored